# Custom exercice: Using TAP to get usable tables for LePHARE

**Learning objective:** Create 2 tables of LSST objects: one for galaxies, one for stars

**LSST data products:** `Object` table

**Packages:** `lsst.rsp.get_tap_service`

## 1. Introduction

TAP (Table Access Protocol) provides standardized access to catalog data for discovery, search, and retrieval.
Full [documentation for TAP](http://www.ivoa.net/documents/TAP) is provided by the
International Virtual Observatory Alliance (IVOA).

### 1.1. TAP glossary

* **schema** - Database terminology for the abstract design that represents the storage of data in a database.
* **TAP schema** - The set of tables that describe the data tables and their columns.
* **table collection** - A collection of tables (catalogs), e.g., for a given data release.
* **table** - A collection of related data held in a table format in a database. 
* **query** - A string formatted in ADQL that selects data from a table, with contraints if desired. 
* **results** - The output of the TAP service's search method when a query is passed.


### 1.2. Reminder: ADQL statements

The [documentation for ADQL](http://www.ivoa.net/documents/latest/ADQL.html)
includes more information about syntax, keywords, operators, functions, and so on.
ADQL is similar to SQL (Structured Query Langage).

A typical ADQL statement has at least three components:

```
SELECT <columns> FROM <catalog> WHERE <constraints>
```

where
 * `<columns>` is a comma-separated list of the columns to return,
 * `<catalog>` is the name of the catalog to retreive data from, and
 * `<constraints>` imposes a restriction that only rows with column values that meet the constraints are returned.

For example, say there is a catalog called "mysurveydata" with 5 columns, "col1", "col2", and so on.
The ADQL statement:

```
SELECT col3, col4, col5 FROM mysurveydata WHERE col1 > 0.5 AND col5 < 10
```

would 
return a table that has three columns, and as many rows as meet both of the restrictions in the `WHERE` statement.

### 1.3. Import packages


In [5]:
import numpy as np
import matplotlib.pyplot as plt

import lsst.geom as geom

from lsst.rsp import get_tap_service
from lsst.rsp.utils import get_pyvo_auth
from lsst.rsp.service import get_siav2_service


## 2. Initialisation of the query

We will be looking at the DP1 catalog schema table to pick columns that would be helpful to identify the source-type. Then we will make our final query to get the right data.

### 2.1 First look at the schema tables.

In [6]:
service = get_tap_service("tap")
assert service is not None

In [7]:
results = service.search('SELECT * FROM tap_schema.schemas')

In [8]:
results.to_table()

schema_name,utype,description,schema_index
str64,str512,str512,int32
dp02_dc2_catalogs,,"Data Preview 0.2 contains the image and catalog products of the Rubin Science Pipelines v23 processing of the DESC Data Challenge 2 simulation, which covered 300 square degrees of the wide-fast-deep LSST survey region over 5 years.",2
dp1,,"Data Preview 1 contains image and catalog products from the Rubin Science Pipelines v29 processing of observations obtained with the LSST Commissioning Camera of seven ~1 square degree fields, over seven weeks in late 2024.",0
ivoa,,ObsCore v1.1 attributes in ObsTAP realization,1
tap_schema,,A TAP-standard-mandated schema to describe tablesets in a TAP 1.1 service,100000


In [9]:
del results

We will obviously chose dp1, but we need to choose from which catalog we will retrieve data. Thus we look at the tap_schema.tables category.

In [19]:
query = "SELECT * FROM tap_schema.tables " \
        "WHERE tap_schema.tables.schema_name = 'dp1'" \
        "ORDER BY table_index ASC"
results = service.search(query).to_table()
results

schema_name,table_name,table_type,utype,description,table_index
str512,str64,str8,str512,str512,int32
dp1,dp1.Object,table,,Descriptions of static astronomical objects (or the static aspects of variable and slowly-moving objects) detected and measured on coadds.,10
dp1,dp1.Source,table,,"Properties of detections on the single-epoch visit images, performed independently of the Object detections on coadded images.",20
dp1,dp1.ForcedSource,table,,"Forced-photometry measurements on individual single-epoch visit images and difference images, based on and linked to the entries in the Object table. Point-source PSF photometry is performed, based on coordinates from a reference band chosen for each Object and reported in the Object.refBand column.",30
dp1,dp1.DiaObject,table,,Properties of time-varying astronomical objects based on association of data from one or more spatially-related DiaSource detections on individual single-epoch difference images.,40
dp1,dp1.DiaSource,table,,Properties of transient-object detections on the single-epoch difference images.,50
dp1,dp1.ForcedSourceOnDiaObject,table,,"Point-source forced-photometry measurements on individual single-epoch visit images and difference images, based on and linked to the entries in the DiaObject table.",60
dp1,dp1.CoaddPatches,table,,Static information about the subset of tracts and patches from the standard LSST skymap that apply to coadds in these catalogs,70
dp1,dp1.Visit,table,,"Metadata about the pointings of the telescope, largely associated with the boresight of the LSSTComCam focal plane as a whole.",80
dp1,dp1.CcdVisit,table,,Metadata about the nine individual CCD images for each Visit in the DP1 LSSTComCam dataset.,90


In [20]:
del query, results

Lets is use the object catalog for this time.

### 2.2 Pick right columns

In the dp1.Object catalog, we need to identify which columns should be useful for LePHARE and later galax

In [21]:
query = "SELECT column_name, datatype, description, unit " \
        "FROM tap_schema.columns " \
        "WHERE table_name = 'dp1.Object'"
results = service.search(query).to_table()
results

column_name,datatype,description,unit
str64,str64,str512,str64
coord_dec,double,Fiducial ICRS Declination of centroid used for database indexing,deg
coord_decErr,float,Error in fiducial ICRS Declination of centroid,deg
coord_ra,double,Fiducial ICRS Right Ascension of centroid used for database indexing,deg
coord_ra_dec_Cov,float,Covariance between fiducial ICRS Right Ascension and Declination of centroid,deg**2
coord_raErr,float,Error in fiducial ICRS Right Ascension of centroid,deg
deblend_failed,boolean,Deblender failed to deblend this source,
deblend_incompleteData,boolean,One or more bands were not deblended due to an inability to model the PSF.,
deblend_isolatedParent,boolean,Deblender skipped this footprint because there was only a single peak,
deblend_iterations,int,Number of iterations during deblending,


In [22]:
search_string = 'ModelMag' #g_cModelMag, g_cModelMagErr
# search_string = 'g_psfFlux'
for cname in results['column_name']:
    if cname.find(search_string) > -1:
        print(cname)

g_cModelMag
g_cModelMagErr
i_cModelMag
i_cModelMagErr
r_cModelMag
r_cModelMagErr
u_cModelMag
u_cModelMagErr
y_cModelMag
y_cModelMagErr
z_cModelMag
z_cModelMagErr


In [23]:
del results

len(results) = 1296 --> we won't need every row

### 2.3. Reminder: useful keywords

We will use a few useful category of keywords to retrieve particular data type:
 * `coord`: ra, dec, errors. Coordinates of the object.
 * `(de)blend`: blendness category includes flag characterizing the mixing of several sources. Perhaps it should be taken into account if blendness and extendedness appeared both as True for a single object
 * `magnitude`: magnitude categories include values associated to ugrizy mag, like numerical value, errors and extendedness. 
 * `{band}_extendedness` where band = ugrizy. This is probably the most useful keywords to identify the source or object type (galaxy or star)

## 3. Trial in the ecdfs region

In [49]:
target_ra = 53.195
target_dec = -27.703
target_r = 1

We won't take into account the AGNs (present in the DIAobject catalog). We will only look at galaxies and stars in the object catalog. To differentiate them efficiently within the LSST catalog, we establish a selection criterion : if 3 bands or more show {band}_extendedness = 1, we'll consider it as a galaxies. Unless the sources will be flagged as stars. But that something that will be useful later.

### 3.1 Full useful parameters table

The first table include every parameters that could be useful in star-galaxies separation, not differentiating the source types.

In [35]:
query = "SELECT objectId, coord_ra, coord_dec, " + \
        "u_cModelMag, u_cModelMagErr, u_extendedness, " + \
        "g_cModelMag, g_cModelMagErr, g_extendedness, " + \
        "r_cModelMag, r_cModelMagErr, r_extendedness, " + \
        "i_cModelMag, i_cModelMagErr, i_extendedness, " + \
        "z_cModelMag, z_cModelMagErr, z_extendedness, " + \
        "y_cModelMag, y_cModelMagErr, y_extendedness, refBand " + \
        "FROM dp1.Object AS obj " + \
        "WHERE (i_cModelFlux/i_cModelFluxErr > 15) AND " + \
        "(u_cModelFlux/u_cModelFluxErr > 5) AND " + \
        "(shape_flag = 0) AND " + \
        "CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
        "CIRCLE('ICRS',"+str(target_ra)+","+str(target_dec)+", "+str(target_r)+")) = 1 "


In [36]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

Job phase is COMPLETED


In [37]:
results = job.fetch_result()
tab = results.to_table()
tab

objectId,coord_ra,coord_dec,u_cModelMag,u_cModelMagErr,u_extendedness,g_cModelMag,g_cModelMagErr,g_extendedness,r_cModelMag,r_cModelMagErr,r_extendedness,i_cModelMag,i_cModelMagErr,i_extendedness,z_cModelMag,z_cModelMagErr,z_extendedness,y_cModelMag,y_cModelMagErr,y_extendedness,refBand
,deg,deg,mag,mag,,mag,mag,,mag,mag,,mag,mag,,mag,mag,,mag,mag,,
int64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str1
611257134226610676,53.10476748835234,-27.518808428034202,22.5108,0.146366,--,20.8857,0.004986,1.0,19.9244,0.00851497,1.0,19.5434,0.00432957,1.0,19.2681,0.0053401,1.0,--,--,--,i
611257134226610953,53.121419521326864,-27.50988041293483,20.2562,0.0271988,1.0,18.4011,0.0011138,1.0,17.3707,0.00182289,1.0,16.9474,0.00087292,1.0,16.6641,0.000832789,1.0,--,--,--,i
611257134226611953,53.128689161074604,-27.48098095098035,22.8901,0.164831,--,22.9038,0.038274,1.0,--,--,--,21.8168,0.044922,--,21.6826,0.04185,1.0,--,--,--,g
611257134226611840,53.092374606157875,-27.485928746442106,23.0955,0.211734,1.0,23.4754,0.0743712,1.0,22.6527,0.0676042,1.0,21.9254,0.0386017,1.0,21.7659,0.0479533,1.0,--,--,--,i
611257134226611818,53.11147971348439,-27.48573926277312,21.1235,0.0467103,--,20.0089,0.00454587,1.0,--,--,--,19.0384,0.00590844,1.0,18.8926,0.00580674,1.0,--,--,--,i
611257134226611819,53.110912371550945,-27.484654013702468,22.0948,0.216328,1.0,22.2102,0.0349323,1.0,--,--,--,21.4348,0.0566617,--,21.4892,0.0879408,--,--,--,--,g
611257134226611188,53.10320303014881,-27.505876157810487,23.0573,0.189803,--,23.2785,0.0320822,1.0,22.7481,0.0652623,1.0,22.5481,0.0463744,1.0,22.7875,0.085957,1.0,--,--,--,i
611257134226610732,53.09457245686273,-27.516850167315347,21.9684,0.0739345,1.0,20.369,0.00257289,1.0,19.3415,0.00453139,1.0,18.9357,0.00215159,1.0,18.6632,0.0023542,1.0,--,--,--,i


### 3.2 Galaxy table

This is already in the short MEME lephare format.

In [71]:
query = "SELECT objectId, " + \
        "u_cModelMag, u_cModelMagErr, " + \
        "g_cModelMag, g_cModelMagErr, " + \
        "r_cModelMag, r_cModelMagErr, " + \
        "i_cModelMag, i_cModelMagErr, " + \
        "z_cModelMag, z_cModelMagErr, " + \
        "y_cModelMag, y_cModelMagErr " + \
        "FROM dp1.Object AS obj " + \
        "WHERE (i_cModelFlux/i_cModelFluxErr > 15) AND " + \
        "(u_cModelFlux/u_cModelFluxErr > 5) AND " + \
        "(shape_flag = 0) AND " + \
        "(g_extendedness = 1) AND " + \
        "(r_extendedness = 1) AND " + \
        "(i_extendedness = 1) AND " + \
        "(z_extendedness = 1) AND " + \
        "CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
        "CIRCLE('ICRS',"+str(target_ra)+","+str(target_dec)+", "+str(target_r)+")) = 1 "

In [72]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

Job phase is COMPLETED


In [73]:
results = job.fetch_result()
tab = results.to_table()
tab

objectId,u_cModelMag,u_cModelMagErr,g_cModelMag,g_cModelMagErr,r_cModelMag,r_cModelMagErr,i_cModelMag,i_cModelMagErr,z_cModelMag,z_cModelMagErr,y_cModelMag,y_cModelMagErr
,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag
int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
611257134226611432,23.0569,0.19202,23.4535,0.036593,23.3484,0.103818,22.9319,0.0519256,22.4067,0.0698534,--,--
611257134226611457,22.7062,0.139444,22.9304,0.021597,22.2742,0.0376351,21.6732,0.016227,21.5039,0.0324328,--,--
611257134226610818,22.9006,0.187118,24.1342,0.0726052,23.5957,0.165865,23.0471,0.0685943,23.0894,0.117306,--,--
611257134226611169,21.231,0.0615021,19.463,0.00201414,18.4514,0.00322382,17.9907,0.00155629,17.7141,0.00213103,--,--
611257134226611170,21.6475,0.0898182,20.0643,0.0030705,19.158,0.00515535,18.7536,0.00262653,18.5165,0.00375925,--,--
611256447031859676,23.3172,0.197396,23.0998,0.0196048,22.9,0.0478193,22.798,0.0416098,22.3281,0.0476338,--,--
611256447031859643,22.5311,0.147581,22.6562,0.0159024,21.6457,0.0201038,20.8032,0.0089626,20.4653,0.0116568,--,--
611256447031840750,22.5936,0.134958,23.3891,0.0429056,22.8435,0.0636949,22.3335,0.0422164,22.1215,0.048837,--,--


In [74]:
job.delete()
del query, results, tab

### 3.2 Star table

In [63]:
query = "SELECT objectId, " + \
        "u_cModelMag, u_cModelMagErr, " + \
        "g_cModelMag, g_cModelMagErr, " + \
        "r_cModelMag, r_cModelMagErr, " + \
        "i_cModelMag, i_cModelMagErr, " + \
        "z_cModelMag, z_cModelMagErr, " + \
        "y_cModelMag, y_cModelMagErr " + \
        "FROM dp1.Object AS obj " + \
        "WHERE (i_cModelFlux/i_cModelFluxErr > 15) AND " + \
        "(u_cModelFlux/u_cModelFluxErr > 5) AND " + \
        "(shape_flag = 0) AND " + \
        "(g_extendedness = 0) AND " + \
        "(r_extendedness = 0) AND " + \
        "(i_extendedness = 0) AND " + \
        "(z_extendedness = 0) AND " + \
        "CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
        "CIRCLE('ICRS',"+str(target_ra)+","+str(target_dec)+", "+str(target_r)+")) = 1 "



In [64]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

Job phase is COMPLETED


In [65]:
results = job.fetch_result()
tab = results.to_table()
tab

objectId,u_cModelMag,u_cModelMagErr,g_cModelMag,g_cModelMagErr,r_cModelMag,r_cModelMagErr,i_cModelMag,i_cModelMagErr,z_cModelMag,z_cModelMagErr,y_cModelMag,y_cModelMagErr
,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag
int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
611257134226610698,22.4926,0.101755,20.0911,0.00206825,18.7267,0.00280399,17.8124,0.00103482,17.4135,0.000830293,--,--
611256515751336084,22.457,0.162976,21.8072,0.00844211,21.5177,0.0169497,21.4217,0.00984476,21.4381,0.0223866,--,--
609788117972428529,17.8141,0.00402551,16.5426,0.000543361,16.0702,0.000852091,15.899,0.00086061,15.8429,0.000721535,--,--
609788117972428638,19.3623,0.00975272,18.2656,0.00109003,17.841,0.00122588,17.6886,0.00149829,17.6594,0.00203953,--,--
609788117972429619,21.6582,0.0553251,19.4215,0.00145514,18.2159,0.001065,16.9896,0.000720644,16.4608,0.000823828,16.2079,0.00131716
609789354923012483,20.7415,0.0106716,18.2119,0.000697807,16.9112,0.000320598,15.9061,0.000436047,15.4808,0.000403378,15.2817,0.00138138
609788667728239946,23.397,0.101236,20.8569,0.00309633,19.6727,0.00133909,19.1329,0.00184654,18.9068,0.0047309,18.752,0.015906
609788667728239945,17.3655,0.00139021,16.2449,0.00026261,15.8344,0.000203986,15.7051,0.000293766,15.6684,0.000572431,15.6729,0.00173974


In [66]:
job.delete()
del query, results, tab

## DC2

In [40]:
query = "SELECT mt.id_truth_type AS mt_id_truth_type, "\
        "mt.match_objectId AS mt_match_objectId, "\
        "ts.ra AS ts_ra, "\
        "ts.dec AS ts_dec, "\
        "ts.truth_type AS ts_truth_type, "\
        "ts.is_pointsource AS ts_is_pointsource, "\
        "ts.redshift AS ts_redshift, "\
        "obj.coord_ra AS obj_coord_ra, "\
        "obj.coord_dec AS obj_coord_dec, "\
        "obj.refExtendedness AS obj_refExtendedness, "\
        "scisql_nanojanskyToAbMag(obj.u_cModelFlux) AS umag, "\
        "scisql_nanojanskyToAbMag(obj.g_cModelFlux) AS gmag, "\
        "scisql_nanojanskyToAbMag(obj.r_cModelFlux) AS rmag, "\
        "scisql_nanojanskyToAbMag(obj.i_cModelFlux) AS imag, "\
        "scisql_nanojanskyToAbMag(obj.z_cModelFlux) AS zmag, "\
        "scisql_nanojanskyToAbMag(obj.y_cModelFlux) AS ymag, "\
        "scisql_nanojanskyToAbMagSigma(obj.g_cModelFlux, obj.u_cModelFluxErr) AS umag_err, "\
        "scisql_nanojanskyToAbMagSigma(obj.g_cModelFlux, obj.g_cModelFluxErr) AS gmag_err, "\
        "scisql_nanojanskyToAbMagSigma(obj.g_cModelFlux, obj.r_cModelFluxErr) AS rmag_err, "\
        "scisql_nanojanskyToAbMagSigma(obj.g_cModelFlux, obj.i_cModelFluxErr) AS imag_err, "\
        "scisql_nanojanskyToAbMagSigma(obj.g_cModelFlux, obj.z_cModelFluxErr) AS zmag_err, "\
        "scisql_nanojanskyToAbMagSigma(obj.g_cModelFlux, obj.y_cModelFluxErr) AS ymag_err "\
        "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
        "JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type "\
        "JOIN dp02_dc2_catalogs.Object AS obj ON mt.match_objectId = obj.objectId "\
        "WHERE CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), CIRCLE('ICRS', 62.0, -37.0, 0.5)) = 1 "\
        "AND scisql_nanojanskyToAbMag(obj.y_cModelFlux)/scisql_nanojanskyToAbMagSigma(obj.y_cModelFlux, obj.y_cModelFluxErr) > 5 "\
        "AND obj.detect_isPrimary = 1"

In [41]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

Job phase is COMPLETED


In [42]:
results = job.fetch_result().to_table().to_pandas()
print(len(results))

389077


In [43]:
results

,mt_id_truth_type,mt_match_objectId,ts_ra,ts_dec,ts_truth_type,ts_is_pointsource,ts_redshift,obj_coord_ra,obj_coord_dec,obj_refExtendedness,...,rmag,imag,zmag,ymag,umag_err,gmag_err,rmag_err,imag_err,zmag_err,ymag_err
0,7940517773_1,1651413688361430386,61.929895,-36.523061,1,0,1.079690,61.929825,-36.523022,1.0,...,26.155623,25.903313,NaN,25.071435,1.011092,0.344147,0.421003,0.789069,2.188033,3.702267
1,7941123620_1,1651413688361451853,61.900438,-36.519216,1,0,1.173830,61.900534,-36.519245,1.0,...,26.445364,27.257660,NaN,25.378260,0.864431,0.278411,0.318666,0.617453,1.674067,3.248848
2,7941880022_1,1651413688361451734,61.869686,-36.521054,1,0,1.301130,61.869544,-36.521018,1.0,...,25.692458,25.894309,24.946288,NaN,0.276842,0.089943,0.106828,0.209841,0.557984,1.067943
3,31411981094_2,1651413688361421449,61.944366,-36.507034,2,1,0.000000,61.944356,-36.507118,1.0,...,25.961597,25.452401,25.984570,24.876525,0.742865,0.249135,0.296967,0.585449,1.546722,2.725028
4,31500767183_2,1651413688361420776,61.926042,-36.520935,2,1,0.000000,61.925989,-36.520908,1.0,...,26.642154,25.257680,25.661938,24.325669,1.139720,0.343763,0.399996,0.766487,2.153979,3.881310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389072,7945365877_1,1651290543059148909,61.901237,-36.882347,1,0,1.819700,61.901248,-36.882354,1.0,...,25.208321,24.954217,24.842578,24.267990,0.120955,0.036944,0.041346,0.080011,0.226193,0.434724
389073,7937556894_1,1651343319617242060,62.214016,-36.764291,1,0,0.223015,62.214047,-36.764303,0.0,...,25.648740,25.602706,25.165273,25.539307,0.324876,0.109264,0.125137,0.245820,0.677128,1.428278
389074,7938263957_1,1651343319617244715,62.077497,-36.705900,1,0,0.624259,62.077490,-36.705880,1.0,...,25.562560,24.937279,24.869845,24.877324,0.359569,0.118269,0.142998,0.271518,0.677786,1.380787
389075,7939442820_1,1651352115710265560,61.992653,-36.734153,1,0,0.942635,61.992615,-36.734149,0.0,...,26.414832,25.482749,25.095093,24.341662,0.684010,0.213506,0.255672,0.494887,1.322551,2.882764
